In [1]:
import json
import numpy as np
import tensorflow as tf

/Users/xenexjoshi/Music Genre Classification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [66]:
DATA_PATH = "data.json"

def load_data(data_path = DATA_PATH):
  with open(data_path, "r") as fp:
    data = json.load(fp)

  mfcc = np.array(data["mfcc"])
  delta_mfcc = np.array(data["delta mfcc"])
  labels = np.array(data["labels"])
  return mfcc, delta_mfcc, labels

In [67]:
from sklearn.model_selection import train_test_split

def split_data(features, labels):
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.30, random_state = 42)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 42)
  return X_train, X_test, X_val, y_train, y_test, y_val

In [68]:
import tensorflow.keras as keras

def generate_cnn_model(input_shape):
  model = keras.Sequential()

  model.add(keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = input_shape))
  model.add(keras.layers.MaxPool2D((3, 3), strides = (2, 2), padding = 'same'))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = input_shape))
  model.add(keras.layers.MaxPool2D((3, 3), strides = (2, 2), padding = 'same'))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(128, (2, 2), activation = 'relu', input_shape = input_shape))
  model.add(keras.layers.MaxPool2D((2, 2), strides = (2, 2), padding = 'same'))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, activation = 'relu'))
  model.add(keras.layers.Dropout(0.3))

  model.add(keras.layers.Dense(10, activation = 'softmax'))
  print("loaded")
  return model

In [69]:
mfcc, delta_mfcc, labels = load_data(DATA_PATH)
print(mfcc.shape)

complex_feature = np.stack([mfcc, delta_mfcc], axis = -1)
print(complex_feature.shape)

(9989, 132, 13)
(9989, 132, 13, 2)


<h3> CNN trained on MFCC: </h3>

In [79]:
# Splitting dataset into training, testing, and validation set
X_train, X_test, X_validation, y_train, y_test, y_validation = split_data(mfcc, labels)

# Adding extra dimension to data for increased stability
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]

print(mfcc.shape)

# Generating input dimension to the CNN
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
print(input_shape)

(5943, 132, 13, 1)
(132, 13, 1)


In [81]:
import tensorflow.keras as keras

# Generating CNN model for input dimension based on MFCC
model_mfcc = generate_cnn_model(input_shape)

# Initializing Adam optimizer for CNN model
optimizer = keras.optimizers.Adam(learning_rate = 0.0001)

# Compiling the model with Adam optimizer
model_mfcc.compile(optimizer = optimizer,
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model_mfcc.summary()

/Users/xenexjoshi/Music Genre Classification/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


loaded


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_68 (Conv2D)              │ (None, 130, 11, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_61 (MaxPooling2D) │ (None, 65, 6, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_60          │ (None, 65, 6, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 63, 4, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_62 (MaxPooling2D) │ (None, 32, 2, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_61          │ (None, 32, 2, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 31, 1, 128)     │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_63 (MaxPooling2D) │ (None, 16, 1, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_62          │ (None, 16, 1, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,394 (720.29 KB)

 Trainable params: 183,946 (718.54 KB)

 Non-trainable params: 448 (1.75 KB)

In [83]:
# Training model on the training set, while using the validation to check validation accuracy after each epoch
model_mfcc.fit(X_train, y_train, validation_data = (X_validation, y_validation), epochs = 20, batch_size = 32)

Epoch 1/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.2628 - loss: 2.3500 - val_accuracy: 0.4090 - val_loss: 1.6518
Epoch 2/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.4748 - loss: 1.5058 - val_accuracy: 0.5434 - val_loss: 1.2742
Epoch 3/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.5657 - loss: 1.2538 - val_accuracy: 0.5815 - val_loss: 1.1757
Epoch 4/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.5949 - loss: 1.1317 - val_accuracy: 0.6101 - val_loss: 1.1142
Epoch 5/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.6262 - loss: 1.0332 - val_accuracy: 0.6292 - val_loss: 1.0673
Epoch 6/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.6563 - loss: 0.9627 - val_accuracy: 0.6530 - val_loss: 1.0215
Epoch 7/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.6790 - loss: 0.8993 - val_accuracy: 0.6473 - val_loss: 1.0214
Epoch 8/20
186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.7135 - loss: 0.8046 - val_accu

In [86]:
# Evaluating the model with the testing set
_, accuracy = model_mfcc.evaluate(X_test, y_test)
print("Model accuracy: " + str(round(100 * accuracy, 3)) + "%.")

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7087 - loss: 0.9104
Model accuracy: 72.272%.
